# Setup

In [1]:
import requests
import json
import pandas as pd
import duckdb

def api_request(endpoint):
    url = f'https://pokeapi.co/api/v2/{endpoint}/'
    response = requests.get(url)
    data = response.json()
    return data

def get_gen_poke(gen):
    data = api_request(f'pokedex/{gen}')
    df = pd.json_normalize(data['pokemon_entries'])
    return df

def get_region_locations(name):
    data = api_request(f'region/{name}')
    df = pd.json_normalize(data['locations'])
    return df

# Main

In [2]:
# 1. Get pokemon from emerald generation
poke_entries = get_gen_poke('hoenn')
poke_entries['id'] = poke_entries['pokemon_species.url'].apply(lambda x: x.split('pokemon-species/')[1].split('/')[0])

# 2. Get emerald region locations
locs = get_region_locations('hoenn')

In [ ]:
# Import data into duckdb
try:
    con = duckdb.connect()
    con.execute("CREATE TABLE pokedex AS SELECT * FROM poke_entries")
finally:
    con.close()

# Debug

In [3]:
locs

,name,url
0,petalburg-city,https://pokeapi.co/api/v2/location/429/
1,slateport-city,https://pokeapi.co/api/v2/location/430/
2,lilycove-city,https://pokeapi.co/api/v2/location/431/
3,mossdeep-city,https://pokeapi.co/api/v2/location/432/
4,sootopolis-city,https://pokeapi.co/api/v2/location/433/
...,...,...
102,terra-cave,https://pokeapi.co/api/v2/location/803/
103,marine-cave,https://pokeapi.co/api/v2/location/804/
104,faraway-island,https://pokeapi.co/api/v2/location/812/
105,hoenn-battle-frontier,https://pokeapi.co/api/v2/location/813/


In [73]:
# Get location encounter data for each location in the Hoenn region
data = api_request('location-area/429')
poke_encounters = pd.json_normalize(
    data['pokemon_encounters'],
    record_path = 'version_details',
    meta = [['pokemon', 'name'], ['pokemon', 'url']]
)

In [69]:
poke_entries

,entry_number,pokemon_species.name,pokemon_species.url,id
0,1,treecko,https://pokeapi.co/api/v2/pokemon-species/252/,252
1,2,grovyle,https://pokeapi.co/api/v2/pokemon-species/253/,253
2,3,sceptile,https://pokeapi.co/api/v2/pokemon-species/254/,254
3,4,torchic,https://pokeapi.co/api/v2/pokemon-species/255/,255
4,5,combusken,https://pokeapi.co/api/v2/pokemon-species/256/,256
...,...,...,...,...
197,198,kyogre,https://pokeapi.co/api/v2/pokemon-species/382/,382
198,199,groudon,https://pokeapi.co/api/v2/pokemon-species/383/,383
199,200,rayquaza,https://pokeapi.co/api/v2/pokemon-species/384/,384
200,201,jirachi,https://pokeapi.co/api/v2/pokemon-species/385/,385
